# Tutorial - Encrypted Inference with ONNX Model

In this tutorial, we will demonstrate how we can run encrypted machine learning inference from a model saved as an [ONNX](https://onnx.ai/) file.

In [2]:
import numpy as np

from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from onnxmltools.convert import convert_sklearn
from skl2onnx.common import data_types as onnx_dtypes

import pymoose as pm
from pymoose.predictors import linear_predictor
from pymoose.predictors import predictor_utils
from pymoose.testing import LocalMooseRuntime

random_state = 5

### Use case: 

You are an HealthCare AI startup who has trained a model to diagnose heart disease. You would like to serve this model to an hospital to help doctors diagnose potential heart disease for their patients. However, the patients' data is too sensitive to be shared with the AI startup. For this reason, you would like to encrypt the patients's data and run the machine learning model on it.

In this tutorial, we will perform the following steps:
- Train a model with [Scikit-Learn](https://scikit-learn.org/stable/)
- Convert the trained model to ONNX.
- Convert the model from ONNX to a Moose computation.
- Run encrypted inference by evaluating the Moose computation.

### Training

For this tutorial, we use a synthetic dataset. The dataset contains 10 features (`X`). Each record is labeled (`y`) by 0 or 1 (heart disasease or not). For the model, we train a logistic regression with [Scikit-Learn](https://scikit-learn.org/stable/). But you could experiment with other models such as `XGBClassifier` from [XGBoost](https://xgboost.readthedocs.io/en/stable/index.html#) or even [Multi-layer Perceptron](https://scikit-learn.org/stable/modules/neural_networks_supervised.html#multi-layer-perceptron).

Once the model is trained, you can convert it to ONNX which is a format to represent machine learning models. Since it's a Scikit-Learn model, you can convert it to ONNX with `convert_sklearn` from [ONNXMLTools](https://github.com/onnx/onnxmltools). Use `convert_xgboost` for XGBoost models.

In [3]:
n_samples = 1000
n_features = 10
n_classes = 2

# Generate synthetic dataset
X, y = make_classification(
    n_samples=n_samples,
    n_features=n_features,
    n_classes=n_classes,
    random_state=random_state,
)

# Split dataset between training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train logistic regression
lg = LogisticRegression()
lg.fit(X_train, y_train)

# Convert scikit-learn model to ONNX
initial_type = ("float_input", onnx_dtypes.FloatTensorType([None, n_features]))
onnx_proto = convert_sklearn(lg, initial_types=[initial_type])

### Convert ONNX to Moose Predictor

PyMoose provides several predictor classes to translate an ONNX model into a PyMoose DSL program.

It currently supports 8 types of model predictors:
- `linear_regressor.LinearRegressor`: for models such as linear regression, ridge regression, etc.
- `linear_regressor.LinearClassifier`: for models such as logistic regression, classifier using ridge regression, etc.
- `tree_ensemble.TreeEnsembleRegressor` for models such as XGBoost regressor, random forest regressor, etc.
- `tree_ensemble.TreeEnsembleClassifier`: for models such as XGBoost classifier, random forest classifier, etc.
- `multilayer_perceptron_predictor.MLPRegressor`: for multi-layer perceptron regressor models. 
- `multilayer_perceptron_predictor.MLPClassifier`: for multi-layer perceptron classifier models.
- `neural_network_predictor.NeuralNetwork`: for feed forward neural network from PyTorch and TensrFlow.

In [4]:
predictor = linear_predictor.LinearClassifier.from_onnx(onnx_proto)

In [11]:
predictor.host_placements

(HostPlacementExpression(name='alice'),
 HostPlacementExpression(name='bob'),
 HostPlacementExpression(name='carole'))

In [12]:
@pm.computation
def moose_predictor_computation(x: pm.Argument(predictor.alice, dtype=pm.float64)):
    with predictor.alice:
        x_fixed = pm.cast(x, dtype=predictor_utils.DEFAULT_FIXED_DTYPE)
    with predictor.replicated:
        y = predictor(x_fixed, predictor_utils.DEFAULT_FIXED_DTYPE)
    return predictor.handle_output(y, prediction_handler=predictor.alice)

In [13]:
traced_predictor = pm.trace(moose_predictor_computation)
storage = {plc.name: {} for plc in predictor.host_placements}
runtime = LocalMooseRuntime(storage_mapping=storage)
role_assignment = {plc.name: plc.name for plc in predictor.host_placements}


result_dict = runtime.evaluate_computation(
    computation=traced_predictor,
    role_assignment=role_assignment,
    arguments={"x": X_test},
)
actual_result = list(result_dict.values())[0]


In [14]:
expected_result = lg.predict_proba(X_test)
np.testing.assert_almost_equal(actual_result, expected_result, decimal=2)